# **Imports**

In [1]:
import pandas as pd

# **Reading Data**

In [2]:
df = pd.read_csv('../Data/combined_data.csv')

# **Data Exploration and Analysis**

In [3]:
df.head()

,Latitude,Longitude,Altitude,Distance,Alignment,Distance_Sensors,Proximity,Wheel_Speed,Velocity,Hall_Effect,Emergency_Stop,Temperature,Humidity,Light_Level
0,37.800175,-121.097884,64.545603,0.0000,False,1.423923,False,2.307947,0.692384,False,False,31.484825,31.292792,548.032945
1,37.800175,-121.097883,64.545603,0.0003,False,2.963388,True,0.790472,0.237141,True,False,21.566685,48.878612,143.937188
2,37.800175,-121.097882,64.545603,0.0006,True,2.989108,True,1.419237,0.425771,False,False,23.682128,40.768480,885.282708
3,37.800175,-121.097881,64.545603,0.0009,False,1.974619,False,3.031683,0.909505,True,False,12.368550,61.984188,828.878655
4,37.800175,-121.097880,64.545603,0.0012,True,3.768380,True,2.644805,0.793441,False,True,12.676042,55.157313,798.250031


In [4]:
df.shape

(50000, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Latitude          50000 non-null  float64
 1   Longitude         50000 non-null  float64
 2   Altitude          50000 non-null  float64
 3   Distance          50000 non-null  float64
 4   Alignment         50000 non-null  bool   
 5   Distance_Sensors  50000 non-null  float64
 6   Proximity         50000 non-null  bool   
 7   Wheel_Speed       50000 non-null  float64
 8   Velocity          50000 non-null  float64
 9   Hall_Effect       50000 non-null  bool   
 10  Emergency_Stop    50000 non-null  bool   
 11  Temperature       50000 non-null  float64
 12  Humidity          50000 non-null  float64
 13  Light_Level       50000 non-null  float64
dtypes: bool(4), float64(10)
memory usage: 4.0 MB


In [6]:
df.describe()

,Latitude,Longitude,Altitude,Distance,Distance_Sensors,Wheel_Speed,Velocity,Temperature,Humidity,Light_Level
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,37.532890,-121.513446,45.886560,0.149850,2.753068,2.735528,0.820658,22.495055,50.094133,499.691181
std,0.287010,0.289524,28.732712,0.086603,1.302263,1.300918,0.390276,7.204729,11.553234,289.635940
min,37.013684,-121.998479,0.575104,0.000000,0.500120,0.500025,0.150008,10.000930,30.000076,0.059913
25%,37.363493,-121.760702,24.731299,0.074925,1.624056,1.608859,0.482658,16.262349,40.094638,248.920004
50%,37.532434,-121.496134,45.878943,0.149850,2.754404,2.720220,0.816066,22.517037,50.162426,500.020790
75%,37.792310,-121.296507,73.942745,0.224775,3.889643,3.862012,1.158604,28.691437,60.184927,750.370008
max,37.997699,-121.015688,99.024772,0.299700,4.999694,4.999887,1.499966,34.999655,69.998836,999.952636


In [7]:
df.isna().sum()

Latitude            0
Longitude           0
Altitude            0
Distance            0
Alignment           0
Distance_Sensors    0
Proximity           0
Wheel_Speed         0
Velocity            0
Hall_Effect         0
Emergency_Stop      0
Temperature         0
Humidity            0
Light_Level         0
dtype: int64

In [8]:
df.columns

Index(['Latitude', 'Longitude', 'Altitude', 'Distance', 'Alignment',
       'Distance_Sensors', 'Proximity', 'Wheel_Speed', 'Velocity',
       'Hall_Effect', 'Emergency_Stop', 'Temperature', 'Humidity',
       'Light_Level'],
      dtype='object')

In [9]:
alignment_conditions = [
    alignment == True for alignment in df["Alignment"]
]

# Adding labels for Chaining Alignment
df["Alignment_Label"] = ["Aligned" if condition else "Not_Aligned" for condition in alignment_conditions]


In [10]:
# Define conditions for safety based on Hall_Effect and Emergency_Stop
safety_conditions = [
    hall_effect == True and emergency_stop == False
    for hall_effect, emergency_stop in zip(
        df["Hall_Effect"],
        df["Emergency_Stop"]
    )
]

# Adding labels for Safety Decision
df["Safety_Label"] = ["Safe" if condition else "Unsafe" for condition in safety_conditions]

In [11]:
# Speed thresholds (in meters per second - m/s)
threshold_high_speed = 1.5  

# Distance thresholds (in meter)
threshold_low_distance = 0.5 

# Temperature thresholds (in degrees Celsius)
threshold_high_temperature = 30 

# Light level thresholds (in Lux)
threshold_low_light = 100


collision_risk_levels = []

for distance, proximity, wheel_speed, temperature, light_level in zip(
    df["Distance"],
    df["Proximity"],
    df["Wheel_Speed"],
    df["Temperature"],
    df["Light_Level"]
):
    if proximity or wheel_speed > threshold_high_speed:
        collision_risk_levels.append(2)
    elif distance < threshold_low_distance:
        collision_risk_levels.append(1)
    elif temperature > threshold_high_temperature or light_level < threshold_low_light:
        collision_risk_levels.append(1)
    else:
        collision_risk_levels.append(0)


# Adding labels for Collision Avoidance
df["Collision_Avoidance_Label"] = ["Low_Risk" if risk_level == 0 else "Medium_Risk" if risk_level == 1 else "High_Risk" for risk_level in collision_risk_levels]

In [12]:
# Define acceptable distance thresholds for chaining 
max_distance = 0.5 
min_distance = 0.15

# Cart wheel speed treshold in meters per second (m/s) for chaining 
high_speed = 2.5

# Define conditions for chaining based on the data
chaining_conditions = [
    alignment and distance > min_distance and distance < max_distance and not proximity and wheel_speed < high_speed and safety_label == "Safe"
    for alignment, distance, proximity, safety_label, wheel_speed in zip(
        df["Alignment"],
        df["Distance"],
        df["Proximity"],
        df["Safety_Label"],
        df['Wheel_Speed']
    )
]

# Adding labels for Chaining/Unchaining Decision
df["Chaining_Unchaining_Label"] = ["Chaining" if condition else "Unchaining" for condition in chaining_conditions]

In [13]:
# Save the updated combined data with labels to a new CSV file
df.to_csv("../Data/final_combined_data.csv", index=False)

print("Combined data with labels saved.")

Combined data with labels saved.


In [14]:
df.columns

Index(['Latitude', 'Longitude', 'Altitude', 'Distance', 'Alignment',
       'Distance_Sensors', 'Proximity', 'Wheel_Speed', 'Velocity',
       'Hall_Effect', 'Emergency_Stop', 'Temperature', 'Humidity',
       'Light_Level', 'Alignment_Label', 'Safety_Label',
       'Collision_Avoidance_Label', 'Chaining_Unchaining_Label'],
      dtype='object')

In [15]:
df.isna().sum()

Latitude                     0
Longitude                    0
Altitude                     0
Distance                     0
Alignment                    0
Distance_Sensors             0
Proximity                    0
Wheel_Speed                  0
Velocity                     0
Hall_Effect                  0
Emergency_Stop               0
Temperature                  0
Humidity                     0
Light_Level                  0
Alignment_Label              0
Safety_Label                 0
Collision_Avoidance_Label    0
Chaining_Unchaining_Label    0
dtype: int64